In [66]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
from imutils.video import VideoStream
import imutils

In [67]:
prototxtPath=os.path.sep.join([r'C:\Learning\Git Repository\Mask_Detection\face_detector','deploy.prototxt'])
weightsPaths=os.path.sep.join([r'C:\Learning\Git Repository\Mask_Detection\face_detector','res10_300x300_ssd_iter_140000.caffemodel'])



In [68]:
prototxtPath

'C:\\Learning\\Git Repository\\Mask_Detection\\face_detector\\deploy.prototxt'

In [69]:
net=cv2.dnn.readNet(prototxtPath,weightsPaths)

In [70]:
model=load_model(r'C:\Learning\Git Repository\Mask_Detection\mobilenet_v2.model')

In [75]:
image=cv2.imread(r'C:\Learning\Git Repository\Mask_Detection\data\with_mask\0-with-mask.jpg')

In [76]:
image

array([[[38, 66, 60],
        [35, 63, 57],
        [32, 59, 50],
        ...,
        [11, 37, 31],
        [18, 41, 36],
        [13, 29, 22]],

       [[35, 63, 57],
        [32, 60, 54],
        [31, 58, 49],
        ...,
        [15, 38, 33],
        [21, 44, 36],
        [15, 31, 24]],

       [[32, 58, 52],
        [31, 57, 51],
        [31, 55, 47],
        ...,
        [17, 40, 32],
        [24, 45, 37],
        [20, 34, 28]],

       ...,

       [[30, 71, 56],
        [31, 72, 57],
        [35, 71, 57],
        ...,
        [12, 12, 18],
        [12, 12, 18],
        [12, 12, 18]],

       [[27, 69, 52],
        [28, 70, 53],
        [32, 68, 54],
        ...,
        [13, 12, 21],
        [13, 12, 21],
        [13, 12, 21]],

       [[26, 68, 51],
        [27, 69, 52],
        [31, 67, 53],
        ...,
        [13, 12, 21],
        [13, 12, 21],
        [13, 12, 21]]], dtype=uint8)

In [78]:
(h,w)=image.shape[:2]

In [79]:
(h,w)

(350, 525)

In [81]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [82]:
blob

array([[[[ -67.,  -73.,  -75., ...,  -74.,  -93.,  -89.],
         [ -71.,  -73.,  -74., ...,  -82.,  -90.,  -86.],
         [ -74.,  -74.,  -74., ...,  -94.,  -87.,  -80.],
         ...,
         [ -73.,  -68.,  -69., ...,  -93.,  -93.,  -93.],
         [ -76.,  -71.,  -71., ...,  -91.,  -91.,  -91.],
         [ -78.,  -73.,  -75., ...,  -91.,  -91.,  -91.]],

        [[-113., -119., -124., ..., -123., -140., -144.],
         [-116., -120., -123., ..., -133., -139., -141.],
         [-121., -123., -123., ..., -147., -138., -137.],
         ...,
         [-105., -105., -109., ..., -165., -166., -166.],
         [-107., -109., -111., ..., -164., -165., -165.],
         [-109., -110., -115., ..., -164., -165., -165.]],

        [[ -65.,  -74.,  -80., ...,  -74.,  -92.,  -96.],
         [ -68.,  -75.,  -79., ...,  -85.,  -91.,  -94.],
         [ -73.,  -77.,  -78., ..., -101.,  -91.,  -90.],
         ...,
         [ -66.,  -65.,  -66., ..., -107., -106., -106.],
         [ -70.,  -69.,  -

In [83]:
blob.shape

(1, 3, 300, 300)

In [84]:
net.setInput(blob)
detections=net.forward()

In [85]:
detections

array([[[[0.        , 1.        , 0.84326315, ..., 0.16504614,
          0.68534595, 0.5733571 ],
         [0.        , 1.        , 0.12824994, ..., 3.9947865 ,
          0.84038293, 4.98498   ],
         [0.        , 1.        , 0.12618425, ..., 3.997875  ,
          4.8332477 , 4.9883804 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [87]:
#loop over the detections
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #include the probability in the label
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
        
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 0s 49ms/step
